# load the dataset MINIST

In [9]:
!pip install -q -U keras-tuner

In [17]:
from tensorflow import keras
from tensorflow.keras import layers
# Load data
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
# Normalize
train_images = train_images.astype("float32") / 255
test_images  = test_images.astype("float32") / 255

# Reshape
train_images = train_images.reshape((60000, 28 * 28))
test_images  = test_images.reshape((10000, 28 * 28))


In [18]:
print(train_images.shape)
print(test_images.shape)

(60000, 784)
(10000, 784)


In [19]:
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [20]:
print(train_labels.shape)
print(test_labels.shape)

(60000, 10)
(10000, 10)


# Build the network model

In [21]:
from keras import models
from keras import layers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
network.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

# Compile the network

In [ ]:
network.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])

# Training dataset : try to "fit" the model

In [ ]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.8719 - loss: 0.4393
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.9665 - loss: 0.1152
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9790 - loss: 0.0728
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9841 - loss: 0.0514
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9892 - loss: 0.0361


# Test data set to evaluate

In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9778 - loss: 0.0734


# Hyperparameter Tuning with Keras Tuner

In [68]:
import keras_tuner as kt
import keras

In [73]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(784,)))

    # Number of hidden layers
    for i in range(hp.Int("num_layers", 2, 10)):
        model.add(
            layers.Dense(
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation="relu"
            )
        )
        model.add(layers.Dropout(
            hp.Float(f"dropout_{i}", 0.2, 0.5, step=0.1)
        ))

    model.add(layers.Dense(10, activation="softmax"))

    learning_rate = hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy", # Changed from sparse_categorical_crossentropy
        metrics=["accuracy"]
    )

    return model

In [81]:
build_model(kt.HyperParameters())

<Sequential name=sequential_2, built=True>

In [80]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=20,
    directory="mnist_tuner",
    project_name="deep_mlp",
    overwrite=True # Added to ensure the latest model definition is used
)

In [83]:
tuner.search(
    train_images, train_labels,
    validation_split=0.1,
    epochs=20,
    batch_size=128, # Changed from kt.Choice to a fixed integer
    callbacks=[
        keras.callbacks.EarlyStopping(patience=3, monitor="accuracy") # Corrected monitor from 'accuarcy' to 'accuracy'
    ]
)

Trial 14 Complete [00h 03m 15s]
val_accuracy: 0.9818333387374878

Best val_accuracy So Far: 0.9848333597183228
Total elapsed time: 01h 51m 18s

Search: Running Trial #15

Value             |Best Value So Far |Hyperparameter
2                 |6                 |num_layers
160               |288               |units_0
0.2               |0.3               |dropout_0
512               |288               |units_1
0.4               |0.2               |dropout_1
0.001             |0.001             |learning_rate
352               |96                |units_2
0.2               |0.4               |dropout_2
192               |352               |units_3
0.2               |0.4               |dropout_3
64                |224               |units_4
0.2               |0.4               |dropout_4
512               |448               |units_5
0.4               |0.3               |dropout_5
32                |320               |units_6
0.2               |0.4               |dropout_6
352              

KeyboardInterrupt: 

In [85]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

print("Best hyperparameters found:")
for key, value in best_hps.values.items():
    print(f"  {key}: {value}")

print("\nSummary of the best model:")
best_model.summary()

Best hyperparameters found:
  num_layers: 6
  units_0: 288
  dropout_0: 0.30000000000000004
  units_1: 288
  dropout_1: 0.2
  learning_rate: 0.001
  units_2: 96
  dropout_2: 0.4
  units_3: 352
  dropout_3: 0.4
  units_4: 224
  dropout_4: 0.4
  units_5: 448
  dropout_5: 0.30000000000000004
  units_6: 320
  dropout_6: 0.4
  units_7: 192
  dropout_7: 0.2
  units_8: 384
  dropout_8: 0.2
  units_9: 256
  dropout_9: 0.2

Summary of the best model:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 288)            │       226,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 288)            │        83,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 96)             │        27,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 352)            │        34,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 352)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 224)            │        79,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 448)            │       100,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │         4,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 555,562 (2.12 MB)

 Trainable params: 555,562 (2.12 MB)

 Non-trainable params: 0 (0.00 B)

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{amsmath}
\usepackage{amssymb}
\usepackage{longtable}

\begin{document}

\section*{Best Hyperparameters Found}
\begin{itemize}
    \item \textbf{num\_layers}: 6
    \item \textbf{units\_0}: 288
    \item \textbf{dropout\_0}: 0.30
    \item \textbf{units\_1}: 288
    \item \textbf{dropout\_1}: 0.2
    \item \textbf{learning\_rate}: 0.001
    \item \textbf{units\_2}: 96
    \item \textbf{dropout\_2}: 0.4
    \item \textbf{units\_3}: 352
    \item \textbf{dropout\_3}: 0.4
    \item \textbf{units\_4}: 224
    \item \textbf{dropout\_4}: 0.4
    \item \textbf{units\_5}: 448
    \item \textbf{dropout\_5}: 0.30
    \item \textbf{units\_6}: 320
    \item \textbf{dropout\_6}: 0.4
    \item \textbf{units\_7}: 192
    \item \textbf{dropout\_7}: 0.2
    \item \textbf{units\_8}: 384
    \item \textbf{dropout\_8}: 0.2
    \item \textbf{units\_9}: 256
    \item \textbf{dropout\_9}: 0.2
\end{itemize}

\section*{Summary of the Best Model}

\begin{longtable}{|l|l|r|}
\hline
\textbf{Layer (type)} & \textbf{Output Shape} & \textbf{Param \#}\\
\hline
\endfirsthead
\hline
\textbf{Layer (type)} & \textbf{Output Shape} & \textbf{Param \#}\\
\hline
\endhead
\hline
\endfoot
\hline
\endlastfoot

dense (Dense) & (None, 288) & 226,080 \\
\hline
dropout (Dropout) & (None, 288) & 0 \\
\hline
dense\_1 (Dense) & (None, 288) & 83,232 \\
\hline
dropout\_1 (Dropout) & (None, 288) & 0 \\
\hline
dense\_2 (Dense) & (None, 96) & 27,744 \\
\hline
dropout\_2 (Dropout) & (None, 96) & 0 \\
\hline
dense\_3 (Dense) & (None, 352) & 34,144 \\
\hline
\end{longtable}

\end{document}
```